### conda prompt → conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
try:
    zipfile.ZipFile('/content/drive/MyDrive/새싹/프로젝트/kb-albert-char-base-v2.zip').extractall()
except zipfile.BadZipFile:
    print("The ZIP file is corrupted or not a valid ZIP file.")

In [4]:
import pandas as pd
import numpy as np
total=pd.read_csv("/content/drive/MyDrive/새싹/프로젝트/df_60.csv", lineterminator = '\n')
total.dropna(inplace=True)
total.set_index(np.arange(len(total)), inplace = True)

In [ ]:
# token n-gram

def div_ngrams(token_list, window_size) :
    li =  [token_list[i : i + window_size] for i in range(len(token_list) - window_size + 1)]
    return [x for x in li if '<START>' not in x and '<END>' not in x]

def get_ngrams(df) :
    df['TOKEN'] = df['TOKEN'].map(lambda x : x.split(" "))
    for i in range(1, 6) :
        df[f'TOKEN_{i}'] = df['TOKEN'].map(lambda x : div_ngrams(x, i))
    return df


#total_report_token = get_ngrams(total)

In [5]:
call_rate=pd.read_csv("/content/drive/MyDrive/새싹/프로젝트/call_rate_polarity(+month).csv")
call_rate["DATE"]=pd.to_datetime(call_rate["DATE"])
call_rate.set_index("DATE", inplace=True, drop=True)
total["DATE"]=pd.to_datetime(total["DATE"])
total_pol=pd.merge(total, call_rate["POLARITY"], how='left', on="DATE")

In [ ]:
!pip install kobert-transformers

In [7]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer
from transformers import AlbertForSequenceClassification
import numpy as np
from torch.optim import AdamW
from copy import deepcopy
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import warnings

In [ ]:
pt_model

In [8]:
kb_albert_model_path = "kb-albert-char-base-v2"
pt_model = AlbertForSequenceClassification.from_pretrained(kb_albert_model_path, num_labels=3)  #라벨 수 3개로 지정
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
pt_model.to(device)

optimizer = AdamW(pt_model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at kb-albert-char-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


라벨 -1,0,1 → 2,0,1

In [ ]:
tokenized_content=tokenizer(total_pol.loc[42, "CLEANED_CONTENTS"].replace("\n", ""), return_tensors='pt', padding="max_length", truncation=True, max_length=512, add_special_tokens=True)
print(tokenized_content["input_ids"])
print(tokenized_content["token_type_ids"])
print(tokenized_content["attention_mask"])

In [11]:
#tokenize 안 한 문장 저장
content_data=[]
content_polarity=[]
max_length=200
for document in tqdm(total_pol.index):
  for sentence in total_pol.loc[document,"CLEANED_CONTENTS"].split("."):
    content_data.append(sentence[:max_length])
    if total_pol.loc[document,"POLARITY_y"]==-1:
      content_polarity.append(2)
    else:
      content_polarity.append(total_pol.loc[document,"POLARITY_y"])

  0%|          | 0/2452 [00:00<?, ?it/s]

In [ ]:
print(tokenized_content["input_ids"][2374])
print(tokenized_content["token_type_ids"][2374])

In [12]:
tokenized_content=tokenizer(content_data, return_tensors='pt', padding="max_length", truncation=True, max_length=128, add_special_tokens=True)
x_train, x_valid, y_train, y_valid=train_test_split(tokenized_content["input_ids"], content_polarity, random_state=np.random.randint(1000000), test_size=0.1)

In [36]:
def calc_accuracy(X,Y):
    pred = torch.argmax(X, dim=1)
    acc = (pred == Y).sum().data.cpu().numpy()/len(Y)
    return acc

In [14]:
batch_size=64
#x_train, x_valid, y_train, y_valid=train_test_split(content_data, content_polarity, random_state=np.random.randint(1000000), test_size=0.1)
train_dataset = TensorDataset(x_train, torch.Tensor(y_train))
valid_dataset = TensorDataset(x_valid, torch.Tensor(y_valid))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
train_dataset

In [ ]:
log_interval=100
loss_history = [] # 전체 loss
avg_loss_hisroty = [] # epoch별 평균 loss
test_history = [] # validation accuracy
test_loss_history = [] # epoch별 평균 validation loss
warnings.filterwarnings("ignore")

best_acc = 0.0
best_acc_model = deepcopy(pt_model)
for epoch in range(10):
    total_loss=0.0
    train_acc = 0.0
    test_acc = 0.0
    test_total_loss=0.0

    pt_model.train() #훈련 선언
    for  batch_id , (token_ids, label) in enumerate(tqdm(iter(train_loader))):
        token_type_ids = torch.LongTensor(np.array([np.array([0]*len(token_id)) for token_id in token_ids])).to(device)
        attention_mask=torch.LongTensor(np.array([np.array([1 if i>0 else 0 for i in token_id]) for token_id in token_ids])).to(device)
        token_ids = torch.tensor(token_ids, dtype=torch.long).to(device)
        label = torch.tensor(label, dtype=torch.long).to(device)

        optimizer.zero_grad()
        pt_outputs = pt_model(token_ids, token_type_ids, attention_mask)[0]
        loss = loss_fn(pt_outputs, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(pt_model.parameters(), 1)
        optimizer.step()

        train_acc += calc_accuracy(pt_outputs, label)

        if (batch_id+1) % log_interval == 0:
            print("[train]\tepoch: {}\t batch id: {}/{}\t train_loss: {:.4f}\t train_acc: {:.4f}".format(epoch+1, batch_id+1,
                int(len(x_train)/batch_size), loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            loss_history.append(loss.data.cpu().numpy())
            total_loss+=loss.data.cpu().numpy()

    avg_loss=total_loss/len(x_train) #epoch별 평균 loss값
    avg_loss_hisroty.append(avg_loss)

    pt_model.eval() #모델평가

    for  batch_id , (token_ids, label) in enumerate(tqdm(iter(valid_loader))):
        label = torch.tensor(label, dtype=torch.long).to(device)
        token_type_ids = torch.LongTensor(np.array([np.array([0]*len(token_id)) for token_id in token_ids])).to(device)
        attention_mask=torch.LongTensor(np.array([np.array([1 if i>0 else 0 for i in token_id]) for token_id in token_ids])).to(device)
        token_ids = torch.tensor(token_ids, dtype=torch.long).to(device)
        with torch.no_grad():
            pt_outputs = pt_model(token_ids, token_type_ids, attention_mask)[0]
            loss = loss_fn(pt_outputs, label).detach()
            test_acc += sum([1 for i, j in zip(torch.argmax(pt_outputs, dim=1), label) if i==j])
            test_total_loss += loss.data.cpu().numpy()
        test_history.append(test_acc / (batch_id+1))

    test_avg_loss=test_total_loss/len(x_valid)
    test_loss_history.append(test_avg_loss)
    print("[valid]\tepoch: {}\t test loss: {:.4f}\t test acc: {:.4f}".format(epoch+1, test_avg_loss,
        test_acc/len(x_valid)))

    if (test_acc / (batch_id+1)) > best_acc:
          best_acc = (test_acc / (batch_id+1))
          best_acc_model = deepcopy(pt_model)

  0%|          | 0/886 [00:00<?, ?it/s]

[train]	epoch: 1	 batch id: 100/885.046875	 train_loss: 1.0423	 train_acc: 0.4359
[train]	epoch: 1	 batch id: 200/885.046875	 train_loss: 1.0102	 train_acc: 0.4335
[train]	epoch: 1	 batch id: 300/885.046875	 train_loss: 1.0262	 train_acc: 0.4352
[train]	epoch: 1	 batch id: 400/885.046875	 train_loss: 1.0701	 train_acc: 0.4341
[train]	epoch: 1	 batch id: 500/885.046875	 train_loss: 1.0745	 train_acc: 0.4296
[train]	epoch: 1	 batch id: 600/885.046875	 train_loss: 1.0406	 train_acc: 0.4286
[train]	epoch: 1	 batch id: 700/885.046875	 train_loss: 1.0410	 train_acc: 0.4265
[train]	epoch: 1	 batch id: 800/885.046875	 train_loss: 0.9976	 train_acc: 0.4267


  0%|          | 0/99 [00:00<?, ?it/s]

[valid]	epoch: 1	 test loss: 0.0164	 test acc: 0.4225


  0%|          | 0/886 [00:00<?, ?it/s]

...

[train]	epoch: 9	 batch id: 100/885.046875	 train_loss: 1.0432	 train_acc: 0.4541
[train]	epoch: 9	 batch id: 200/885.046875	 train_loss: 1.0430	 train_acc: 0.4548
[train]	epoch: 9	 batch id: 300/885.046875	 train_loss: 1.0287	 train_acc: 0.4551
[train]	epoch: 9	 batch id: 400/885.046875	 train_loss: 0.9924	 train_acc: 0.4521
[train]	epoch: 9	 batch id: 500/885.046875	 train_loss: 1.0215	 train_acc: 0.4536
[train]	epoch: 9	 batch id: 600/885.046875	 train_loss: 1.0681	 train_acc: 0.4541
[train]	epoch: 9	 batch id: 700/885.046875	 train_loss: 1.0043	 train_acc: 0.4545
[train]	epoch: 9	 batch id: 800/885.046875	 train_loss: 1.0194	 train_acc: 0.4550


  0%|          | 0/99 [00:00<?, ?it/s]

[valid]	epoch: 9	 test loss: 0.0162	 test acc: 0.4504


  0%|          | 0/886 [00:00<?, ?it/s]

[train]	epoch: 10	 batch id: 100/885.046875	 train_loss: 1.0267	 train_acc: 0.4328


### 예측

In [15]:
def convert_data(sent):
    tokenized_sent=tokenizer(sent, return_tensors='pt', padding="max_length", truncation=True, max_length=128, add_special_tokens=True)
    return tokenized_sent

def test_sentences(sent):
    pt_model.eval()
    input=convert_data(sent)
    input=input.to(device)
    with torch.no_grad():
        output=pt_model(**input)[0]
        pred=np.argmax(output)

    if pred==2:
        result=-1
    else:
        result=int(pred)

    return result


In [17]:
minute_df.head()

,TITLE,DATE,CONTENTS,CLEANED_CONTENTS
0,금융통화위원회 의사록(2023년도 제14차)(2023.7.27),2023.7.27,"['가. 의결안건', '<의안 제31호 – ｢한국은행의 금융기관대출규정｣ 개정(안)...",가. 의결안건\n의안 제31호 한국은행의 금융기관대출규정 개정\n의안 제32호 한국...
1,금융통화위원회 의사록(2023년도 제13차)(2023.7.13),2023.7.13,"['가. 의결안건', '〈의안 제30호 ― 통화정책방향〉(1) 전일 개최된 동향보고...",가. 의결안건\n의안 제30호 통화정책방향 전일 개최된 동향보고회의에서는 조사국장이...
2,금융통화위원회 의사록(2023년도 제12차)(2023.6.21),2023.6.21,"['가. 의결안건', '<의안 제28호 – 금융안정보고서(2023년 6월)(안)>'...",가. 의결안건\n의안 제28호 금융안정보고서 2023년 6월\n담당 부총재보가 한국...
3,금융통화위원회 의사록(2023년도 제11차)(2023.6.8),2023.6.8,"['가. 의결안건', '<의안 제25호 – 통화신용정책보고서(2023년 6월)(안)...",가. 의결안건\n의안 제25호 통화신용정책보고서 2023년 6월\n담당 부총재보가 ...
4,금융통화위원회 의사록(2023년도 제10차)(2023.5.25),2023.5.25,"['가. 의결안건', '〈의안 제24호 ― 통화정책방향〉(1) 전일 개최된 동향보고...",가. 의결안건\n의안 제24호 통화정책방향 전일 개최된 동향보고회의에서는 조사국장이...


In [ ]:
minute_df=pd.read_csv("/content/drive/MyDrive/새싹/프로젝트/의사록_cleaned.csv") #아직 안 돌려봤는데 이런식으로 한 문장씩 긍정부정 판별한 다음에 문서 tone 구하면 될 듯?
haw_tone=0
dov_tone=0
document_tone=[]
for mpb in range(len(minute_df)):
  for sentence in tqdm(minute_df.loc[mpb,"CLEANED_CONTENTS"].split("\n")):
    if test_sentences(sentence) == 1:
      haw_tone+=1
    elif test_sentences(sentence) == -1:
      dov_tone+=1
  document_tone.append(1 if (haw_tone-dov_tone)/(haw_tone+dov_tone)>0 else -1)

In [ ]:
document_tone

### 수많은 시행착오들

In [ ]:
from sklearn.model_selection import train_test_split
input_transformer = InputTransformer(tokenizer, 1000)
batch_size=64
trn_sents, valid_sents, trn_labels, valid_labels = train_test_split(content_data, content_polarity, test_size=0.1, random_state=np.random.randint(1000000))

train_dataset = BERTDataset(input_transformer, trn_sents, trn_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0)

valid_dataset = BERTDataset(input_transformer, valid_sents, valid_labels)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=0)

In [ ]:
tokenizer.convert_tokens_to_ids('[SEP]')

3

In [ ]:
class InputTransformer:

    def __init__(self, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_token = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]

    def transform(self, sentences):
        tokenized = [self.tokenizer.tokenizetokenizer(sentence, return_tensors='pt', padding=True, truncation=True) for sentence in sentences]
        input_ids = [self.tokenizer.convert_tokens_to_ids(['[CLS]'] + tokens + ['[SEP]'])[:self.max_len] for tokens in tokenized]
        valid_lengths = [input_id.index(3) if len(input_id) < self.max_len else self.max_len for input_id in input_ids]
        token_ids = [np.array(input_id + [self.pad_token]*(self.max_len-valid_lengths[i]-1)) for i, input_id in enumerate(input_ids)]
        segment_ids = [np.zeros(self.max_len) for i in range(len(sentences))]

        return list(zip(token_ids, valid_lengths, segment_ids))

In [ ]:
class BERTDataset(Dataset):

    def __init__(self, transformer, sentences, labels=()):
        self.sentences = transformer.transform(sentences)
        self.is_test = False if labels else True
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __getitem__(self, i):
        return self.sentences[i], self.labels[i] if not self.is_test else _

    def __len__(self):
        return len(self.sentences)

### dataset x

In [ ]:
log_interval=100
loss_history = [] # 전체 loss
avg_loss_hisroty = [] # epoch별 평균 loss
test_history = [] # validation accuracy
test_loss_history = [] # epoch별 평균 validation loss

best_acc = 0.0
best_acc_model = deepcopy(pt_model)
for e in range(10):
    total_loss=0.0
    train_acc = 0.0
    test_acc = 0.0
    test_total_loss=0.0

    pt_model.train() #훈련 선언
    for  batch_id , (sentence, label) in enumerate(zip(tqdm(x_train), y_train)):
      optimizer.zero_grad()
      pt_outputs = pt_model(**sentence.to(device))[0]
      label = torch.LongTensor([label]).to(device)
      loss = loss_fn(pt_outputs, label)
      loss.backward()
      optimizer.step()

      train_acc += calc_accuracy(pt_outputs, label)

      if (batch_id+1) % log_interval == 0:
            print("[train]\tepoch: {}\t batch id: {}/{}\t train_loss: {:.4f}\t train_acc: {:.4f}".format(e+1, batch_id+1,
                len(x_train), loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            loss_history.append(loss.data.cpu().numpy())
            total_loss+=loss.data.cpu().numpy()

    avg_loss=total_loss/len(x_train) #epoch별 평균 loss값
    avg_loss_hisroty.append(avg_loss)

    pt_model.eval() #모델평가
    for  batch_id , (sentence, label) in enumerate(zip(tqdm(x_valid), y_valid)):
        label = torch.LongTensor([label]).to(device)
        with torch.no_grad():
            pt_outputs = pt_model(**sentence.to(device))[0]
            loss = loss_fn(pt_outputs, label).detach()
            test_acc += calc_accuracy(pt_outputs, label)
            test_total_loss += loss.data.cpu().numpy()
        test_history.append(test_acc / (batch_id+1))

    test_avg_loss=test_total_loss/len(x_valid)
    test_loss_history.append(test_avg_loss)
    print("[valid]\tepoch: {}\t test loss: {:.4f}\t test acc: {:.4f}".format(e+1, test_avg_loss,
        test_acc/len(x_valid)))

    if (test_acc / (batch_id+1)) > best_acc:
          best_acc = (test_acc / (batch_id+1))
          best_acc_model = deepcopy(pt_model)

In [ ]:
total_pol.loc[1786,"CLEANED_CONTENTS"]

In [ ]:
pt_model.eval()
for i in total_pol.loc[1786,"CLEANED_CONTENTS"].split("."):
  pt_inputs = tokenizer(i, return_tensors='pt', padding=True, truncation=True)
  pt_outputs = pt_model(**pt_inputs)[0]
print(torch.argmax(pt_outputs, dim=1), pt_outputs)

tensor([2]) tensor([[0.0943, 0.0053, 0.1103]], grad_fn=<AddmmBackward0>)


In [ ]:
total_pol.loc[1,"CLEANED_CONTENTS"]

'독일프랑크푸르트 소재 유럽중앙은행 ECB 유로화 마크 조형물 AP 연합뉴스DB 영국 중앙은행도 금리 0.50퍼센트 유지 베를린 연합뉴스 박창욱 특파원 유럽중앙은행 ECB 4일 현지시간 기준금리를 현행 0.50퍼센트로 동결했다.\nECB는 독일 프랑크푸르트 본부에서 정례 금융통화정책 회의를 열어 이같이 결정했다고 발표했다.\nECB는 지난해 7월 기준금리를 0.25퍼센트 포인트 내린 이후 9개월간 동결 기조를 유지해오다가 지난 5월 0.25퍼센트 포인트 인하한 있다.\nECB는 이날 현행 0.5퍼센트인 하루짜리 최저 대출금리와 0퍼센트인 초단기 예금금리도 동결했다.\n이날 금리 동결은 시장의 전망과 일치하는 것이다.\n그래픽 유럽중앙은행 ECB 기준금리 추이 서울 연합뉴스 유럽중앙은행 ECB 4일 현지시간 기준금리를 현행 0.50퍼센트로 동결했다 yonhap_graphics 트위터 포르투갈의 정국 불안으로 유로존 금융위기가 다시 고개를 조짐을 보이고 있으나 독일을 중심으로 유로존의 최근 경제 지표들이 다소 개선됨에 따라 일단 상황을 지켜보자는 것이 ECB 내부의 주된 기류로 관측됐다.\n특히 지난달 미국 연방준비제도 연준 양적완화 축소를 시사한 이후 세계 금융시장에서 출구전략 우려가 커진 상태이기 때문에 섣불리 금리 정책의 변화를 모색하는 것은 혼란을 가중할 있는 상황이다.\n마리오 드라기 ECB 총재는 회견에서 금리 동결 배경과 향후 통화정책 방향에 관해 설명하는 한편 포르투갈 문제에 대해서도 언급할 것으로 알려졌다.\n이날 영국 중앙은행도 기준 금리를 현행 0.50퍼센트로 묶고 자산매입 규모를 3천750억 파운드로 유지하기로다.'

In [ ]:
input=tokenizer("경제 분야에서는 중국이 위안화 평가 절하 기조를 이어가고 영국중앙은행은 기준금리 인상을 단행하지 않을 것이라고 FT는 점쳤다.", return_tensors="pt",, padding="max_length", truncation=True, max_length=256)
output=pt_model(**input)[0]
print(torch.argmax(outputs, dim=1), outputs)

SyntaxError: ignored

In [ ]:

for i in total_pol.loc[1,"CLEANED_CONTENTS"].split("."):
  pt_inputs = tokenizer(i, return_tensors='pt').to(device)
  pt_outputs = pt_model(**pt_inputs)[0]
  print(torch.argmax(pt_outputs, dim=1), pt_outputs)

tensor([2], device='cuda:0') tensor([[ 0.0975, -0.1990,  0.2470]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([2], device='cuda:0') tensor([[-0.0070, -0.1550,  0.2828]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([2], device='cuda:0') tensor([[-0.0820, -0.2078,  0.1920]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([2], device='cuda:0') tensor([[-0.2957, -0.1765, -0.0242]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([2], device='cuda:0') tensor([[-0.1212, -0.2940,  0.1724]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([0], device='cuda:0') tensor([[ 0.2175, -0.1466,  0.1119]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([2], device='cuda:0') tensor([[-0.0034, -0.0006,  0.3080]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([2], device='cuda:0') tensor([[-0.0588, -0.2808,  0.0917]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([1], device='cuda:0') tensor([[-0.0843,  0.0701,  0.0361]

In [ ]:
print(pt_inputs["input_ids"].shape)
print(pt_inputs["token_type_ids"].shape)
print(pt_inputs["attention_mask"])


In [ ]:
input_ids[0]

### tokenize 안 쓰고 df["TOKEN"] 바로 넣어서 돌리기

In [ ]:
input_ids = torch.LongTensor(np.array(tokenizer.convert_tokens_to_ids([i.split("/")[0] for i in total_report_token.loc[1,"TOKEN"]])).reshape(1,-1))
token_type_ids = torch.LongTensor(np.array([0]*len(total_report_token.loc[1,"TOKEN"])).reshape(1,-1))
attention_mask=torch.Tensor(np.array([1 if i>0 else 0 for i in input_ids[0]]).reshape(1,-1))
input_ids={token_type_ids}
pt_outputs = pt_model(**input_ids)[0]

In [ ]:
pt_outputs

In [ ]:
pred=torch.argmax(pt_outputs, dim=1)
pred

In [ ]:
!pip3 install kobert-transformers

In [ ]:
!wget https://raw.githubusercontent.com/monologg/KoBERT-Transformers/master/kobert_transformers/tokenization_kobert.py -O ./tokenization_kobert.py

In [ ]:
from kobert_transformers import get_kobert_model, get_distilkobert_model
from tokenization_kobert import KoBertTokenizer

In [ ]:
tokenizer=KoBertTokenizer.from_pretrained("monologg/kobert")
tokenizer.tokenize(total_report_token.loc[1,"CLEANED_CONTENTS"])

In [ ]:
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

In [ ]:

from transformers import  pipeline

nsmc_classifier = pipeline("sentiment-analysis", model=pt_model, tokenizer=tokenizer, framework="pt")
reviews = ["이 영화 최악이었어!", "볼거리가 많은 내 인생 영화 ㅎㅎ"]
results = nsmc_classifier(reviews)
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")